In [1]:
import pandas as pd
import numpy as np

In [2]:
d = pd.read_html("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M", header = 0)
df = d[0]

In [3]:
# Ignore cells that is Not assigned.
df['Borough'].replace('Not assigned', np.nan, inplace=True)
df.dropna(subset=['Borough'], inplace=True)

In [5]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough
for i in range(len(df)):
    if df['Neighbourhood'].iloc[i] == 'Not assigned':
        df['Neighbourhood'].iloc[i] =  df['Borough'].iloc[i]

In [6]:
# combining the neighborhoods into one line separated by a comma
df = df.groupby(['Postcode', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df.head(12)

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
5,M1J,Scarborough,Scarborough Village
6,M1K,Scarborough,"East Birchmount Park, Ionview, Kennedy Park"
7,M1L,Scarborough,"Clairlea, Golden Mile, Oakridge"
8,M1M,Scarborough,"Cliffcrest, Cliffside, Scarborough Village West"
9,M1N,Scarborough,"Birch Cliff, Cliffside West"


First cell involves needed imports<br>
Second cell involves the webpage scraping code<br>
In the third cell 'Not assigned' in Borough column fixed<br>
In the fourth cell 'Not assigned' Neighbourhood column filled with its Borough column<br>
In the fourth cell Neigbourhoods are combined according to Postcode and Borough<br>

In [7]:
df.shape

(103, 3)